In [180]:
import numpy as np
import random

In [181]:
# resposta esperada pra cada padrão
response_vectors = np.array([
    [1, -1, -1, -1, -1, -1, -1, -1],
    [-1, 1, -1, -1, -1, -1, -1, -1],
    [-1, -1, 1, -1, -1, -1, -1, -1],
    [-1, -1, -1, 1, -1, -1, -1, -1],
    [-1, -1, -1, -1, 1, -1, -1, -1],
    [-1, -1, -1, -1, -1, 1, -1, -1],
    [-1, -1, -1, -1, -1, -1, 1, -1],
    [-1, -1, -1, -1, -1, -1, -1, 1]
], dtype=np.float32)

def one_hot_encode(x):
    return response_vectors[x]

def add_noise():
    return round(random.randint(0,1) + random.uniform(-0.1, 0.1),4) # adiciona ruído

total_data = 1000
data_X = np.zeros((total_data,3), dtype=np.float32)
data_y = np.zeros((total_data,8), dtype=np.float32)

for i in range(total_data):
    val = np.array([add_noise(), add_noise(), add_noise()])
    data_X[i] = val

    code = np.array([round(val[0]), round(val[1]), round(val[2])])
    label = np.array([code[0]*4 + code[1]*2 + code[2]])
    data_y[i] = one_hot_encode(label)

train_X, val_X = np.split(data_X, 2)
train_y, val_y = np.split(data_y, 2)

val_X, test_X = np.split(val_X, 2)
val_y, test_y = np.split(val_y, 2)



In [182]:
def constroi_listaPCI(X:np.array, y:np.array, w:np.array) -> tuple:
    signs = np.sign(X.dot(w))
    predicted_labels = np.argmax(signs, axis=1)
    actual_labels = np.argmax(y, axis=1)
    incorrect_indices = predicted_labels != actual_labels
    PCI = X[incorrect_indices]
    PCI_y = y[incorrect_indices]

    return PCI, PCI_y

def PLA(X:np.array, y:np.array, max_iter=10000) -> np.array:
    w = np.zeros((3,8))
    listaPCI = X.copy()
    label = y.copy()
    iter_count = 0
    
    while len(listaPCI) > 0 and iter_count < max_iter:
        i = np.random.randint(len(listaPCI))
        x = listaPCI[i]

        w += np.outer(x, label[i])

        listaPCI, label = constroi_listaPCI(X, y, w)
        iter_count += 1
    return w


In [183]:
w = PLA(train_X, train_y)

val_predictions = np.sign(val_X.dot(w))
accuracy = np.mean(val_predictions == val_y)
print(accuracy * 100, '%')

82.39999999999999 %


In [184]:
import pickle

with open('classification_model.pkl', 'wb') as f:
    pickle.dump(w, f)
